In [ ]:
import pandas as pd
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 경로 설정
file_path = '/content/pone.0197032.s001.xlsx'  # 정확한 파일명 사용

# 데이터 불러오기
data = pd.read_excel(file_path)

# 데이터 출력
import IPython.display as display
display.display(data.head())  # 상위 5개의 데이터만 보기


Saving pone.0197032.s001.xlsx to pone.0197032.s001.xlsx


,Sex_1male_2female,Age,Weather_1sunny_2cloudy_3rainy_4missing,Location_1outdoor_indoor,Unnamed: 4,HT,HeartDisease,Pscyco,DM,CerevD,...,Plt,BUN,Cre,AST,ALT,CK,CRP,Admission1,ICU,Deadtodischarge
0,1.0,24.0,3.0,NaN,1.0,0,0,0,0,0,...,23.2,17.0,1.69,29.0,48.0,506.0,17.4,1.0,NaN,NaN
1,NaN,43.0,NaN,1.0,NaN,0,0,0,0,0,...,24.3,23.1,3.08,36.0,35.0,883.0,0.46,1.0,NaN,NaN
2,1.0,58.0,NaN,1.0,1.0,0,0,0,0,0,...,31.2,19.4,1.42,33.0,16.0,815.0,0.16,1.0,NaN,NaN
3,1.0,46.0,1.0,1.0,1.0,0,0,0,0,0,...,33.3,32.9,4.20,33.0,54.0,173.0,1,1.0,NaN,NaN
4,1.0,57.0,1.0,1.0,1.0,0,0,0,0,0,...,14.1,21.0,1.33,77.0,36.0,386.0,0.103,1.0,1.0,NaN


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 데이터 불러오기
data = pd.read_excel('/content/pone.0197032.s001.xlsx')

# 문자열에서 숫자만 추출하는 함수
def convert_to_number(val):
    try:
        return float(val)
    except ValueError:
        return np.nan  # 변환 불가능한 경우 NaN으로 처리

# 데이터 전처리
data = data.iloc[:2500]  # 1~2500 행만 사용
data = data.dropna(subset=['Admission1'])  # 'Admission1' 컬럼에 NaN 값이 있으면 제거

# 모든 데이터에 대해 숫자만 추출
data = data.applymap(convert_to_number)

# 숫자형 데이터만 선택 (int, float)
data = data.select_dtypes(include=[float, int])

# 'Admission1'을 타겟 변수로 설정
X = data.drop(columns=['Admission1'])
y = data['Admission1'].astype(int)  # 'Admission1'을 타겟 변수로 설정

# 특성 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PyTorch 텐서로 변환
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

# 훈련 데이터셋 준비
train_data = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# 딥러닝 모델 정의
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 2)  # 출력이 2개 (온열질환환자: 0, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 모델, 손실 함수, 최적화 알고리즘 정의
input_dim = X_tensor.shape[1]  # 입력 차원
model = SimpleNN(input_dim)
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

<ipython-input-3-859dd54e496e>:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(convert_to_number)


Epoch 1/20, Loss: nan
Epoch 2/20, Loss: nan
Epoch 3/20, Loss: nan
Epoch 4/20, Loss: nan
Epoch 5/20, Loss: nan
Epoch 6/20, Loss: nan
Epoch 7/20, Loss: nan
Epoch 8/20, Loss: nan
Epoch 9/20, Loss: nan
Epoch 10/20, Loss: nan
Epoch 11/20, Loss: nan
Epoch 12/20, Loss: nan
Epoch 13/20, Loss: nan
Epoch 14/20, Loss: nan
Epoch 15/20, Loss: nan
Epoch 16/20, Loss: nan
Epoch 17/20, Loss: nan
Epoch 18/20, Loss: nan
Epoch 19/20, Loss: nan
Epoch 20/20, Loss: nan


In [ ]:
# 2501~3176행 테스트 데이터 준비
test_data = pd.read_excel('/content/pone.0197032.s001.xlsx').iloc[2500:]  # 2501~3176 행
test_data = test_data.applymap(convert_to_number)

# 숫자형 데이터만 선택 (int, float)
test_data = test_data.select_dtypes(include=[float, int])

# 'Admission1' 컬럼에 NaN 값이 있을 수 있으므로 이를 처리
test_data = test_data.dropna(subset=['Admission1'])  # 'Admission1'에 NaN 값이 있으면 제거

X_test = test_data.drop(columns=['Admission1'])
y_test = test_data['Admission1'].astype(int)  # 'Admission1'을 타겟 변수로 설정

# 테스트 데이터 전처리
X_test_scaled = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# 테스트 데이터에 대한 예측
model.eval()  # 모델을 평가 모드로 전환
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# 정확도 계산
accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 62.45%


<ipython-input-5-fbf1abde9a8b>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data = test_data.applymap(convert_to_number)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 데이터 불러오기
data = pd.read_excel('/content/pone.0197032.s001.xlsx')

# 문자열에서 숫자만 추출하는 함수
def convert_to_number(val):
    try:
        return float(val)
    except ValueError:
        return np.nan  # 변환 불가능한 경우 NaN으로 처리

# 데이터 전처리
data = data.iloc[:2500]  # 1~2500 행만 사용
data = data.dropna(subset=['Admission1'])  # 'Admission1' 컬럼에 NaN 값이 있으면 제거

# 모든 데이터에 대해 숫자만 추출
data = data.applymap(convert_to_number)

# 숫자형 데이터만 선택 (int, float)
data = data.select_dtypes(include=[float, int])

# 'Admission1'을 타겟 변수로 설정
X = data.drop(columns=['Admission1'])
y = data['Admission1'].astype(int)  # 'Admission1'을 타겟 변수로 설정

# 특성 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PyTorch 텐서로 변환
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

# 훈련 데이터셋 준비
train_data = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# 딥러닝 모델 정의
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 2)  # 출력이 2개 (온열질환환자: 0, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 모델, 손실 함수, 최적화 알고리즘 정의
input_dim = X_tensor.shape[1]  # 입력 차원
model = SimpleNN(input_dim)
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습 (에포크 수 늘리기)
epochs = 50  # 에포크 수
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # 훈련 데이터 학습
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # 훈련 정확도 계산
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # 훈련 정확도
    train_accuracy = 100 * correct_train / total_train

    # 2501~3176행 테스트 데이터 준비
    test_data = pd.read_excel('/content/pone.0197032.s001.xlsx').iloc[2500:]  # 2501~3176 행
    test_data = test_data.applymap(convert_to_number)

    # 숫자형 데이터만 선택 (int, float)
    test_data = test_data.select_dtypes(include=[float, int])

    # 'Admission1' 컬럼에 NaN 값이 있을 수 있으므로 이를 처리
    test_data = test_data.dropna(subset=['Admission1'])  # 'Admission1'에 NaN 값이 있으면 제거

    X_test = test_data.drop(columns=['Admission1'])
    y_test = test_data['Admission1'].astype(int)  # 'Admission1'을 타겟 변수로 설정

    # 테스트 데이터 전처리
    X_test_scaled = scaler.transform(X_test)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    # 테스트 데이터에 대한 예측
    model.eval()  # 모델을 평가 모드로 전환
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)

    # 정확도 계산
    correct_test = (predicted == y_test_tensor).sum().item()
    total_test = y_test_tensor.size(0)
    test_accuracy = 100 * correct_test / total_test

    # 에포크마다 훈련 및 테스트 정확도 출력
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')


<ipython-input-6-6f50427dc741>:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(convert_to_number)
<ipython-input-6-6f50427dc741>:92: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data = test_data.applymap(convert_to_number)


Epoch 1/50, Train Loss: nan, Train Accuracy: 60.56%, Test Accuracy: 62.45%


<ipython-input-6-6f50427dc741>:92: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data = test_data.applymap(convert_to_number)


Epoch 2/50, Train Loss: nan, Train Accuracy: 60.73%, Test Accuracy: 62.45%


<ipython-input-6-6f50427dc741>:92: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data = test_data.applymap(convert_to_number)


...

<ipython-input-6-6f50427dc741>:92: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data = test_data.applymap(convert_to_number)


Epoch 9/50, Train Loss: nan, Train Accuracy: 60.73%, Test Accuracy: 62.45%


<ipython-input-6-6f50427dc741>:92: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data = test_data.applymap(convert_to_number)


Epoch 10/50, Train Loss: nan, Train Accuracy: 60.73%, Test Accuracy: 62.45%


KeyboardInterrupt: 

1. 데이터 전처리
  - 공백 처리
  - 중간 화살표 문자열 처리. 데이터 전처리
    - 공백 처리
      - 중간 화살표 문자열 처리